In [1]:
import cv2
import numpy as np 
import argparse
import time
import pandas as pd

In [2]:
def load_yolo():
    net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
    classes = []
    with open("coco.names", "r") as f:
        classes = [line.strip() for line in f.readlines()] 

    output_layers = [layer_name for layer_name in net.getUnconnectedOutLayersNames()]
    colors = np.random.uniform(0, 255, size=(len(classes), 3))
    return net, classes, colors, output_layers

In [3]:
def load_image(img_path):
    # image loading
    img = cv2.imread(img_path)
    img = cv2.resize(img, None, fx=0.4, fy=0.4)
    height, width, channels = img.shape
    return img, height, width, channels

In [4]:
def detect_objects(img, net, outputLayers):
    blob = cv2.dnn.blobFromImage(img, scalefactor=0.00392, size=(320, 320), mean=(0, 0, 0), swapRB=True, crop=False)
    net.setInput(blob)
    outputs = net.forward(outputLayers)
    return blob, outputs

In [5]:
def get_box_dimensions(outputs, height, width):
    boxes = []
    confs = []
    class_ids = []
    for output in outputs:
        for detect in output:
            scores = detect[5:]
            class_id = np.argmax(scores)
            conf = scores[class_id]
            if conf > 0.5:
                center_x = int(detect[0] * width)
                center_y = int(detect[1] * height)
                w = int(detect[2] * width)
                h = int(detect[3] * height)
                x = int(center_x - w/2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confs.append(float(conf))
                class_ids.append(class_id)
    return boxes, confs, class_ids

In [6]:
def draw_labels(boxes, confs, colors, class_ids, classes, img): 
    indexes = cv2.dnn.NMSBoxes(boxes, confs, 0.5, 0.4)
    font = cv2.FONT_HERSHEY_PLAIN
    
    person=0
    car=0
    suitcase=0
    traffic=0
    handbag=0
    maxi=[]
    timestamp = []
    for i in indexes.flatten():
        cmax=0
        labels = str(classes[class_ids[i]])
        if labels == 'person':
            person+=1
        if labels == 'car':
            car+=1
        if labels == 'suitcase':
            suitcase+=1
        if labels == 'traffic light':
            traffic+=1
        if labels == 'handbag':
            handbag+=1
        cmax = person + car + suitcase + traffic + handbag
        maxi.append(cmax)
    
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            color = colors[i]
            cv2.rectangle(img, (x,y), (x+w, y+h), color, 2)
            cv2.putText(img, label, (x, y - 5), font, 1, color, 1)
            cv2.putText(img,'Person'+":"+str(person),(20,20), font, 0.8, (0,0,0), 2)
            cv2.putText(img,'car'+":"+str(car),(20,50), font, 0.8, (0,0,0), 2)
            cv2.putText(img,'Suitcase'+":"+str(suitcase),(20,80), font, 0.8, (0,0,0), 2)
            cv2.putText(img,'Traffic light'+":"+str(traffic),(20,110), font, 0.8, (0,0,0), 2)
            cv2.putText(img,'Handbag'+":"+str(handbag),(20,140), font, 0.8, (0,0,0), 2)
            
    out.write(img)        
    cv2.imshow("Image", img)
    return max(maxi)

In [7]:
model, classes, colors, output_layers = load_yolo()
cap = cv2.VideoCapture('pedestrians.mp4')

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH) + 0.5)
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT) + 0.5)
size = (width, height)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('pedestrians_output1.avi', fourcc, 20.0, size)
maxi=0
maximum = []
timestamp=[]
while True:
    istrue, frame = cap.read()
    if istrue:
        height, width, channels = frame.shape
        blob, outputs = detect_objects(frame, model, output_layers)
        boxes, confs, class_ids = get_box_dimensions(outputs, height, width)
        #print("Toy Detected at: "+str(cap.get(cv2.CAP_PROP_POS_frames)))
        maxi = draw_labels(boxes, confs, colors, class_ids, classes, frame)
        maximum.append(maxi)
        timestamp.append(str(cap.get(cv2.CAP_PROP_POS_MSEC)/1000))
        if cv2.waitKey(1) & 0xFF==ord('q'):
            break
    else:
        break
        
print("Maximum detection of objects at a particular frame is ", max(maximum))  

cap.release()
cv2.destroyAllWindows()

Maximum detection of objects at a particular frame is  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


In [9]:
maximum

[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 [1, 2, 3, 4, 5, 6, 7, 8, 9,

In [8]:
timestamp_of_maximum_object_detection = []

for i , j in zip(maximum, timestamp):
    if i == max(maximum):
        timestamp_of_maximum_object_detection.append(j)

final_table = pd.DataFrame()        
final_table['Maximum Object detection at seconeds '] = timestamp_of_maximum_object_detection
print(final_table)

  Maximum Object detection at seconeds 
0                   0.45879166666666665
1                    1.5849166666666665
